In [1]:
import os
import shutil
from sklearn.model_selection import train_test_split

def create_train_test_dirs(base_path):
    """Create train and test directories within the base path"""
    train_dir = os.path.join(base_path, 'train')
    test_dir = os.path.join(base_path, 'test')
    
    for dir_path in [train_dir, test_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
    
    return train_dir, test_dir

def split_data(base_path, split_ratio=0.7):
    """Split data from base path into train and test sets"""
    # Get all image files from base directory
    image_files = [f for f in os.listdir(base_path) 
                  if f.endswith(('.jpg', '.jpeg', '.png')) 
                  and os.path.isfile(os.path.join(base_path, f))]
    
    # Create train/test directories
    train_dir, test_dir = create_train_test_dirs(base_path)
    
    # Split the data
    train_files, test_files = train_test_split(image_files, 
                                             train_size=split_ratio, 
                                             random_state=42)
    
    # Move files to respective directories
    for file in train_files:
        src = os.path.join(base_path, file)
        dst = os.path.join(train_dir, file)
        shutil.move(src, dst)
    
    for file in test_files:
        src = os.path.join(base_path, file)
        dst = os.path.join(test_dir, file)
        shutil.move(src, dst)
    
    print(f"Total images: {len(image_files)}")
    print(f"Training images: {len(train_files)}")
    print(f"Testing images: {len(test_files)}")

if __name__ == "__main__":
    # Set your base path where images are located
    base_path = "dataset"
    
    # Split and move data
    split_data(base_path, split_ratio=0.7)

In [2]:
import os
import shutil
import random
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
train_path = os.path.join("dataset", "train")  # Path to training data
augmented_path = os.path.join("dataset", "train_augmented")

os.makedirs(augmented_path, exist_ok=True)

def get_next_image_number(train_path):
    """Find the highest image number in the existing files and return next number"""
    existing_files = [f for f in os.listdir(train_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
    numbers = []
    
    for f in existing_files:
        try:
            # Extract number from filenames like 'IMG_102.jpg'
            num = int(f.split('_')[1].split('.')[0])
            numbers.append(num)
        except (IndexError, ValueError):
            continue

    return max(numbers) + 1 if numbers else 102  # Start numbering from 102 if no images exist

# Get list of training images
train_files = [f for f in os.listdir(train_path) if f.lower().endswith(('png', 'jpg', 'jpeg'))]
next_image_number = get_next_image_number(train_path)

# Define augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

# Function to save augmented images
def save_augmented_image(image, number):
    filename = f'IMG_{number:03d}.jpg'
    cv2.imwrite(os.path.join(augmented_path, filename), image)

# Apply augmentation
num_augmented = 0
for file in train_files:
    img_path = os.path.join(train_path, file)
    image = cv2.imread(img_path)
    
    if image is None:
        continue  # Skip invalid images

    image = np.expand_dims(image, axis=0)  # Expand dimensions to match ImageDataGenerator input

    # Generate augmented images
    aug_iter = datagen.flow(image, batch_size=1)

    for i in range(5):  # Generate 5 augmented versions per image
        aug_image = next(aug_iter)[0].astype(np.uint8)
        save_augmented_image(aug_image, next_image_number)
        next_image_number += 1
        num_augmented += 1

print(f'Generated {num_augmented} augmented images in {augmented_path}')

# Move augmented images to training directory
for aug_file in os.listdir(augmented_path):
    src = os.path.join(augmented_path, aug_file)
    dst = os.path.join(train_path, aug_file)
    shutil.move(src, dst)

# Remove temporary augmented directory
os.rmdir(augmented_path)

print(f'Moved all augmented images to training directory')

In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import warnings  # Add this import
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.backends.mps
from torch.utils.data import Dataset, DataLoader
from torch.nn import functional as F
# Fix for circular import - import specific components instead of full modules
from torchvision import transforms
from torchvision.models import resnet50
from torchvision.transforms import ToTensor, Resize, Normalize, Compose
from tqdm import tqdm

In [4]:
# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Attention Block for focusing on relevant features
class AttentionBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // 8, kernel_size=1),
            nn.BatchNorm2d(in_channels // 8),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // 8, in_channels, kernel_size=1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        attention_weights = self.attention(x)
        return x * attention_weights

In [5]:
# ASPP Module for multi-scale feature extraction
class ASPPModule(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 1)
        self.conv3_1 = nn.Conv2d(in_channels, out_channels, 3, padding=1, dilation=1)
        self.conv3_6 = nn.Conv2d(in_channels, out_channels, 3, padding=6, dilation=6)
        self.conv3_12 = nn.Conv2d(in_channels, out_channels, 3, padding=12, dilation=12)
        self.bn = nn.BatchNorm2d(out_channels * 4)
        self.conv1x1 = nn.Conv2d(out_channels * 4, out_channels, 1)
        
    def forward(self, x):
        conv1 = self.conv1(x)
        conv3_1 = self.conv3_1(x)
        conv3_6 = self.conv3_6(x)
        conv3_12 = self.conv3_12(x)
        concat = torch.cat([conv1, conv3_1, conv3_6, conv3_12], dim=1)
        return self.conv1x1(self.bn(concat))

In [6]:
# Improved Segmentation Model
class ImprovedSegmentationModel(nn.Module):
    def __init__(self, n_classes=1):
        super().__init__()
        # Change the model initialization
        resnet = resnet50(weights='IMAGENET1K_V1')
        
        # Encoder
        self.encoder1 = nn.Sequential(*list(resnet.children())[:3])
        self.encoder2 = nn.Sequential(*list(resnet.children())[3:5])
        self.encoder3 = list(resnet.children())[5]
        self.encoder4 = list(resnet.children())[6]
        self.encoder5 = list(resnet.children())[7]
        
        # Attention blocks
        self.attention1 = AttentionBlock(256)
        self.attention2 = AttentionBlock(512)
        self.attention3 = AttentionBlock(1024)
        
        # ASPP module
        self.aspp = ASPPModule(2048, 256)
        
        # Decoder
        self.decoder4 = nn.Sequential(
            nn.ConvTranspose2d(256, 256, kernel_size=2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True)
        )
        
        self.decoder3 = nn.Sequential(
            nn.ConvTranspose2d(1280, 128, kernel_size=2, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True)
        )
        
        self.decoder2 = nn.Sequential(
            nn.ConvTranspose2d(640, 64, kernel_size=2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True)
        )
        
        self.decoder1 = nn.Sequential(
            nn.ConvTranspose2d(320, 32, kernel_size=2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True)
        )
        
        self.final_conv = nn.Sequential(
            nn.Conv2d(32, 16, kernel_size=3, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
            nn.Conv2d(16, n_classes, kernel_size=1),
            nn.Sigmoid()
        )
        
        self._initialize_weights()
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x):
        # Encoding
        e1 = self.encoder1(x)
        e2 = self.encoder2(e1)
        e3 = self.encoder3(e2)
        e4 = self.encoder4(e3)
        e5 = self.encoder5(e4)
        
        # Apply attention
        a2 = self.attention1(e2)
        a3 = self.attention2(e3)
        a4 = self.attention3(e4)
        
        # Apply ASPP
        aspp_out = self.aspp(e5)
        
        # Decoding with skip connections and attention
        d4 = self.decoder4(aspp_out)
        d4_concat = torch.cat([d4, a4], dim=1)
        
        d3 = self.decoder3(d4_concat)
        d3_concat = torch.cat([d3, a3], dim=1)
        
        d2 = self.decoder2(d3_concat)
        d2_concat = torch.cat([d2, a2], dim=1)
        
        d1 = self.decoder1(d2_concat)
        
        # Final convolution
        out = self.final_conv(d1)
        
        # Resize to input size
        out = F.interpolate(out, size=x.shape[2:], mode='bilinear', align_corners=True)
        
        return out

In [7]:
# Replace CombinedLoss with DiceLoss
class DiceLoss(nn.Module):
    def __init__(self, smooth=1e-5):
        super().__init__()
        self.smooth = smooth
        
    def forward(self, pred, target):
        intersection = torch.sum(pred * target)
        union = torch.sum(pred) + torch.sum(target)
        dice = (2.0 * intersection + self.smooth) / (union + self.smooth)
        return 1.0 - dice

In [8]:
def calculate_metrics(pred, target):
    """Calculate Accuracy, IoU, Precision, Recall, F1 Score"""
    pred = (pred > 0.5).float()
    
    tp = torch.sum(pred * target)
    fp = torch.sum(pred * (1 - target))
    fn = torch.sum((1 - pred) * target)
    tn = torch.sum((1 - pred) * (1 - target))
    
    accuracy = (tp + tn) / (tp + fp + fn + tn + 1e-6)
    precision = tp / (tp + fp + 1e-6)
    recall = tp / (tp + fn + 1e-6)
    f1 = 2 * (precision * recall) / (precision + recall + 1e-6)
    iou = tp / (tp + fp + fn + 1e-6)
    
    return {
        'accuracy': accuracy.item(),
        'precision': precision.item(),
        'recall': recall.item(),
        'f1': f1.item(),
        'iou': iou.item()
    }

def calculate_iou(pred, target):
    """Calculate IoU score"""
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    union = pred.sum() + target.sum() - intersection
    return (intersection + 1e-6) / (union + 1e-6)

def calculate_dice_score(pred, target):
    """Calculate Dice score"""
    pred = (pred > 0.5).float()
    intersection = (pred * target).sum()
    return (2. * intersection + 1e-6) / (pred.sum() + target.sum() + 1e-6)

def get_training_config():
    return {
        'batch_size': 8,  # Increased batch size for faster processing
        'num_workers': 2,  # Added some workers for data loading
        'pin_memory': True,
        'prefetch_factor': 2,  # Add prefetching for better data loading
    }

def plot_metrics(metrics_history, output_folder):
    """Plot training and validation metrics"""
    metrics = ['loss', 'accuracy', 'precision', 'recall', 'f1', 'iou']
    plt.figure(figsize=(20, 15))
    
    for idx, metric in enumerate(metrics, 1):
        plt.subplot(3, 2, idx)
        plt.plot(metrics_history['train'][metric], label=f'Train {metric}')
        plt.plot(metrics_history['val'][metric], label=f'Val {metric}')
        plt.title(f'{metric.capitalize()} over epochs')
        plt.xlabel('Epoch')
        plt.ylabel(metric.capitalize())
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'training_metrics.png'))
    plt.close()

In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs, device, output_folder):
    best_val_metrics = {'dice': 0.0, 'f1': 0.0}
    metrics_history = {
        'train': {'loss': [], 'accuracy': [], 'dice': [], 'precision': [], 'recall': [], 'f1': [], 'iou': []},
        'val': {'loss': [], 'accuracy': [], 'dice': [], 'precision': [], 'recall': [], 'f1': [], 'iou': []}
    }
    
    # Create progress logger with additional metrics
    metrics_file = os.path.join(output_folder, 'training_metrics.csv')
    with open(metrics_file, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['Epoch', 'Train_Loss', 'Train_Accuracy', 'Train_Dice', 'Train_Precision', 
                        'Train_Recall', 'Train_F1', 'Train_IoU', 'Val_Loss', 'Val_Accuracy',
                        'Val_Dice', 'Val_Precision', 'Val_Recall', 'Val_F1', 'Val_IoU'])
    
    # Enable cuDNN autotuner
    torch.backends.cudnn.benchmark = True
    
    for epoch in range(num_epochs):
        model.train()
        train_metrics = {'loss': 0, 'accuracy': 0, 'dice': 0, 'precision': 0, 'recall': 0, 'f1': 0, 'iou': 0}
        
        # Modified progress bar with all metrics
        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}')
        
        for batch_idx, (images, masks) in enumerate(progress_bar):
            images, masks = images.to(device), masks.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            # Calculate all metrics for this batch
            dice_score = 1 - loss.item()
            batch_metrics = calculate_metrics(outputs, masks)
            
            # Update metrics
            train_metrics['loss'] += loss.item()
            train_metrics['accuracy'] += batch_metrics['accuracy']
            train_metrics['dice'] += dice_score
            train_metrics['precision'] += batch_metrics['precision']
            train_metrics['recall'] += batch_metrics['recall']
            train_metrics['f1'] += batch_metrics['f1']
            train_metrics['iou'] += batch_metrics['iou']
            
            loss.backward()
            optimizer.step()
            
            # Update progress bar with all metrics
            progress_bar.set_postfix({
                'loss': f"{loss.item():.4f}",
                'accuracy': f"{batch_metrics['accuracy']:.4f}",
                'dice': f"{dice_score:.4f}",
                'prec': f"{batch_metrics['precision']:.4f}",
                'rec': f"{batch_metrics['recall']:.4f}",
                'f1': f"{batch_metrics['f1']:.4f}",
                'iou': f"{batch_metrics['iou']:.4f}"
            })
        
        # Calculate average training metrics
        num_batches = len(train_loader)
        for k in train_metrics:
            train_metrics[k] /= num_batches
        
        # Validation phase
        model.eval()
        val_metrics = {'loss': 0, 'accuracy': 0, 'dice': 0, 'precision': 0, 'recall': 0, 'f1': 0, 'iou': 0}
        
        with torch.no_grad():
            for images, masks in val_loader:
                images, masks = images.to(device), masks.to(device)
                outputs = model(images)
                loss = criterion(outputs, masks)
                
                # Calculate metrics
                dice_score = 1 - loss.item()
                batch_metrics = calculate_metrics(outputs, masks)
                
                # Update metrics
                val_metrics['loss'] += loss.item()
                val_metrics['accuracy'] += batch_metrics['accuracy']
                val_metrics['dice'] += dice_score
                val_metrics['precision'] += batch_metrics['precision']
                val_metrics['recall'] += batch_metrics['recall']
                val_metrics['f1'] += batch_metrics['f1']
                val_metrics['iou'] += batch_metrics['iou']
        
        # Calculate average validation metrics
        num_val_batches = len(val_loader)
        for k in val_metrics:
            val_metrics[k] /= num_val_batches
        
        # Update learning rate
        scheduler.step(val_metrics['loss'])
        
        # Save metrics
        with open(metrics_file, 'a', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                epoch + 1,
                f"{train_metrics['loss']:.4f}",
                f"{train_metrics['accuracy']:.4f}",
                f"{train_metrics['dice']:.4f}",
                f"{train_metrics['precision']:.4f}",
                f"{train_metrics['recall']:.4f}",
                f"{train_metrics['f1']:.4f}",
                f"{train_metrics['iou']:.4f}",
                f"{val_metrics['loss']:.4f}",
                f"{val_metrics['accuracy']:.4f}",
                f"{val_metrics['dice']:.4f}",
                f"{val_metrics['precision']:.4f}",
                f"{val_metrics['recall']:.4f}",
                f"{val_metrics['f1']:.4f}",
                f"{val_metrics['iou']:.4f}"
            ])
        
        # Print epoch summary
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print(f"Train - Loss: {train_metrics['loss']:.4f}, Accuracy: {train_metrics['accuracy']:.4f}, Dice: {train_metrics['dice']:.4f}, "
              f"F1: {train_metrics['f1']:.4f}, Precision: {train_metrics['precision']:.4f}, "
              f"Recall: {train_metrics['recall']:.4f}, IoU: {train_metrics['iou']:.4f}")
        print(f"Val - Loss: {val_metrics['loss']:.4f}, Accuracy: {val_metrics['accuracy']:.4f}, Dice: {val_metrics['dice']:.4f}, "
              f"F1: {val_metrics['f1']:.4f}, Precision: {val_metrics['precision']:.4f}, "
              f"Recall: {val_metrics['recall']:.4f}, IoU: {val_metrics['iou']:.4f}")
        
    
        print(f"Saved new best model with dice score: {val_metrics['dice']:.4f}")
        
        # Store metrics history
        for phase in ['train', 'val']:
            metrics = train_metrics if phase == 'train' else val_metrics
            for k in metrics:
                metrics_history[phase][k].append(metrics[k])
    
    # After training completes, plot the metrics
    plot_metrics(metrics_history, output_folder)
    return metrics_history

def create_binary_masks(image_path):
    """Create binary masks using traditional CV method"""
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    lower_bound = np.array([35, 40, 20])
    upper_bound = np.array([85, 255, 255])
    mask = cv2.inRange(image, lower_bound, upper_bound)
    return mask

In [10]:
# Dataset Class
class VegetationDataset(Dataset):
    def __init__(self, image_paths, masks=None, transform=None):
        self.image_paths = image_paths
        self.masks = masks
        self.transform = transform
        # Load images immediately instead of using multiprocessing
        self.loaded_images = {}
        self.loaded_masks = {}
        self._preload_images()
    
    def _preload_images(self):
        for idx, path in enumerate(self.image_paths):
            image = cv2.imread(path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            self.loaded_images[idx] = image
            
            if self.masks is not None:
                self.loaded_masks[idx] = cv2.resize(self.masks[idx], (320, 320))
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = self.loaded_images[idx]
        
        if self.transform:
            image = self.transform(image)
        
        if self.masks is not None:
            mask = self.loaded_masks[idx]
            mask = torch.FloatTensor(mask / 255.0)
            mask = mask.unsqueeze(0)
            return image, mask
        return image

In [11]:
def preprocess_image(image_path, model, device, transform):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original_size = image.shape[:2]
    
    if transform:
        image_tensor = transform(image).unsqueeze(0).to(device)
    
    with torch.no_grad():
        mask_pred = model(image_tensor)
        mask_pred = mask_pred.squeeze().cpu().numpy()
        mask_pred = cv2.resize(mask_pred, (original_size[1], original_size[0]))
        mask_pred = (mask_pred > 0.5).astype(np.uint8) * 255
    
    return image, mask_pred

def segment_image(image, mask):
    segmented = cv2.bitwise_and(image, image, mask=mask)
    return mask, segmented

def estimate_carbon_sequestration(mask, pixel_area=0.01):  # pixel_area in square meters
    """
    Calculate carbon sequestration based on vegetation area and biomass estimation
    Args:
        mask: Binary mask of vegetation
        pixel_area: Area represented by each pixel in square meters
    Returns:
        Dictionary containing carbon sequestration metrics
    """
    vegetation_area = np.sum(mask > 0) * pixel_area  # Total area in square meters
    
    # Biomass calculation constants
    biomass_factor = 0.5  # Average biomass per square meter for vegetation
    carbon_fraction = 0.47  # Standard carbon fraction in biomass
    co2_conversion = 3.67  # Conversion factor from C to CO2
    
    # Calculate metrics
    total_biomass = vegetation_area * biomass_factor  # kg
    carbon_content = total_biomass * carbon_fraction  # kg C
    co2_sequestered = carbon_content * co2_conversion  # kg CO2
    
    return {
        'vegetation_area_m2': vegetation_area,
        'biomass_kg': total_biomass,
        'carbon_content_kg': carbon_content,
        'co2_sequestered_kg': co2_sequestered
    }

In [12]:
def create_visualizations(all_visualizations, output_folder):
    """
    Create and save visualizations with enhanced carbon sequestration metrics
    """
    num_images = len(all_visualizations)
    fig = plt.figure(figsize=(20, 5 * num_images))
    
    for idx, (image, mask, segmented, metrics) in enumerate(all_visualizations):
        # Original image
        plt.subplot(num_images, 4, idx*4 + 1)
        plt.title(f"Original {idx+1}")
        plt.imshow(image)
        plt.axis('off')
        
        # Mask
        plt.subplot(num_images, 4, idx*4 + 2)
        plt.title(f"Vegetation Mask {idx+1}")
        plt.imshow(mask, cmap='gray')
        plt.axis('off')
        
        # Segmented image
        plt.subplot(num_images, 4, idx*4 + 3)
        plt.title(f"Segmented Vegetation {idx+1}")
        plt.imshow(segmented)
        plt.axis('off')
        
        # Metrics text
        plt.subplot(num_images, 4, idx*4 + 4)
        plt.axis('off')
        plt.text(0.1, 0.7, 
                f"Area: {metrics['vegetation_area_m2']:.2f} m²\n"
                f"Biomass: {metrics['biomass_kg']:.2f} kg\n"
                f"Carbon: {metrics['carbon_content_kg']:.2f} kg C\n"
                f"CO₂ Seq: {metrics['co2_sequestered_kg']:.2f} kg CO₂",
                fontsize=10, fontfamily='monospace')
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'all_results.png'), dpi=300, bbox_inches='tight')
    plt.close()
    
    # Save individual results
    individual_dir = os.path.join(output_folder, 'individual_results')
    os.makedirs(individual_dir, exist_ok=True)
    
    for idx, (image, mask, segmented, metrics) in enumerate(all_visualizations):
        # Save metrics to CSV
        metrics_file = os.path.join(individual_dir, f'metrics_{idx+1}.csv')
        with open(metrics_file, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['Metric', 'Value'])
            for key, value in metrics.items():
                writer.writerow([key, f'{value:.2f}'])
        
        # Save images
        plt.figure(figsize=(15, 5))
        
        plt.subplot(1, 3, 1)
        plt.imshow(image)
        plt.title(f"Original {idx+1}")
        plt.axis('off')
        
        plt.subplot(1, 3, 2)
        plt.imshow(mask, cmap='gray')
        plt.title(f"Mask {idx+1}")
        plt.axis('off')
        
        plt.subplot(1, 3, 3)
        plt.imshow(segmented)
        plt.title(f"Segmented {idx+1}")
        plt.axis('off')
        
        plt.savefig(os.path.join(individual_dir, f'result_{idx+1}.png'), 
                   dpi=300, bbox_inches='tight')
        plt.close()

In [ ]:
def setup_device():
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS backend for Mac GPU acceleration")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA GPU acceleration")
    else:
        device = torch.device("cpu")
        print("Using CPU")
    return device

def get_training_config():
    return {
        'batch_size': 8,  # Increased batch size for faster processing
        'num_workers': 2,  # Added some workers for data loading
        'pin_memory': True,
        'prefetch_factor': 2,  # Add prefetching for better data loading
    }

# Create data loaders with simplest configuration
def create_data_loaders(train_dataset, val_dataset, train_config):
    train_loader = DataLoader(
        train_dataset,
        batch_size=train_config['batch_size'],
        shuffle=True,
        num_workers=0,
        pin_memory=train_config['pin_memory']
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=train_config['batch_size'],
        num_workers=0,
        pin_memory=train_config['pin_memory']
    )
    
    return train_loader, val_loader

def get_transform():
    return Compose([
        ToTensor(),
        Resize((320, 320), antialias=True),
        Normalize(mean=[0.485, 0.456, 0.406], 
                 std=[0.229, 0.224, 0.225])
    ])

In [ ]:
def main():
    # Setup paths
    train_data_folder = 'dataset/train'  # Path to training data
    val_data_folder = 'dataset/test'     # Add missing val_data_folder
    output_folder = 'result'             # Add missing output_folder
    os.makedirs(output_folder, exist_ok=True)
    
    # Get device
    device = setup_device()
    print(f"Using device: {device}")
    
    train_config = get_training_config()
    
    # Data preparation with pre-split data
    print("Preparing dataset...")
    train_image_paths = []
    train_masks = []
    val_image_paths = []
    val_masks = []
    
    # Load training data
    for file_name in os.listdir(train_data_folder):
        if file_name.lower().endswith(('.jpg', '.png')):
            file_path = os.path.join(train_data_folder, file_name)
            mask = create_binary_masks(file_path)
            train_image_paths.append(file_path)
            train_masks.append(mask)
    
    # Load validation data
    for file_name in os.listdir(val_data_folder):
        if file_name.lower().endswith(('.jpg', '.png')):
            file_path = os.path.join(val_data_folder, file_name)
            mask = create_binary_masks(file_path)
            val_image_paths.append(file_path)
            val_masks.append(mask)
    
    # Optimized transform for M1
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((320, 320), antialias=True),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                           std=[0.229, 0.224, 0.225])
    ])
    
    # Create optimized datasets
    train_dataset = VegetationDataset(train_image_paths, train_masks, transform=transform)
    val_dataset = VegetationDataset(val_image_paths, val_masks, transform=transform)
    
    # Create data loaders using the helper function
    train_loader, val_loader = create_data_loaders(train_dataset, val_dataset, train_config)
    
    # Ensure model weights are on correct device
    model = ImprovedSegmentationModel()
    model = model.to(device)  # Simplified device transfer
    
    # Ensure model weights are on correct device
    for param in model.parameters():
        param.data = param.data.to(device)
    
    # Initialize training components with pure Dice Loss
    criterion = DiceLoss()
    optimizer = optim.AdamW(
        model.parameters(),
        lr=0.001,  # Slightly higher learning rate
        weight_decay=0.01,
        betas=(0.9, 0.999)
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.5,
        patience=3,
        verbose=True,
        min_lr=1e-6
    )
    
    # Training
    print("Starting training...")
    # Train the model
    metrics_history = train_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        criterion=criterion,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=100,
        device=device,
        output_folder=output_folder
    )
    
    
    # Process test images
    print("Processing test images...")
    test_images = []
    
    # Process all test images (remove the break condition)
    for file_name in os.listdir(val_data_folder):
        if file_name.lower().endswith(('.jpg', '.png')):
            file_path = os.path.join(val_data_folder, file_name)
            image, mask_pred = preprocess_image(file_path, model, device, transform)
            mask, segmented = segment_image(image, mask_pred)
            
            metrics = estimate_carbon_sequestration(mask)
            test_images.append((image, mask, segmented, metrics))
            
            print(f"\nFile: {file_name}")
            print(f"Vegetation Area: {metrics['vegetation_area_m2']:.2f} m²")
            print(f"Estimated Biomass: {metrics['biomass_kg']:.2f} kg")
            print(f"Carbon Content: {metrics['carbon_content_kg']:.2f} kg C")
            print(f"CO₂ Sequestered: {metrics['co2_sequestered_kg']:.2f} kg CO₂")
    
    # Create visualizations for all images
    if test_images:
        create_visualizations(test_images, output_folder)
        print(f"\nVisualization and metrics saved to {output_folder}")
    else:
        print("No test images were processed.")

if __name__ == "__main__":
    main()

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read the CO2 sequestration data
df = pd.read_csv('result/CO2_seq.csv')

# Function to determine trend and additional metrics
def analyze_trend(values, window_size=20):
    # Remove NaN values
    values = values.dropna()
    
    # Basic trend
    x = np.arange(len(values))
    coefficients = np.polyfit(x, values, 1)
    trend = 'INCREASING' if coefficients[0] > 0 else 'DECREASING'
    
    # Calculate short-term trends using rolling windows
    rolling_means = values.rolling(window=window_size).mean()
    short_term_trends = rolling_means.diff().apply(lambda x: 'INCREASING' if x > 0 else 'DECREASING')
    
    return {
        'Overall_Trend': trend,
        'Rate_of_Change': coefficients[0],
        'Current_Value': values.iloc[-1],
        'Mean': values.mean(),
        'Max': values.max(),
        'Min': values.min(),
        'Std_Dev': values.std(),
        'Short_Term_Trends': short_term_trends
    }

# Calculate metrics for different aspects
time_periods = df.index
trend_data = []

# Analyze different metrics
metrics = {
    'Carbon_Sequestration': df['carbon_sequestration'],
    'Moving_Average': df['carbon_sequestration'].rolling(window=20).mean(),
    'Running_Total': df['carbon_sequestration'].cumsum(),
    'Running_Average': df['carbon_sequestration'].expanding().mean()
}

# Analyze each metric
for metric_name, values in metrics.items():
    analysis = analyze_trend(values)
    
    # Create trend data for each time period
    for period in time_periods:
        trend_data.append({
            'Time_Period': period,
            'Metric': metric_name,
            'Value': values.iloc[period] if period < len(values) else None,
            'Overall_Trend': analysis['Overall_Trend'],
            'Rate_of_Change': analysis['Rate_of_Change'],
            'Short_Term_Trend': analysis['Short_Term_Trends'].iloc[period] if period < len(analysis['Short_Term_Trends']) else None
        })

# Create DataFrame and save to CSV
trend_df = pd.DataFrame(trend_data)
trend_df.to_csv('result/trend_analysis.csv', index=False)

# Create visualizations with trend indicators
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, (metric_name, values) in enumerate(metrics.items()):
    ax = axes[idx]
    analysis = analyze_trend(values)
    
    # Plot main values
    ax.plot(values, label=metric_name)
    
    # Add trend line
    x = np.arange(len(values.dropna()))
    trend_line = analysis['Rate_of_Change'] * x + np.polyfit(x, values.dropna(), 1)[1]
    ax.plot(x, trend_line, '--', 
           label=f"Trend ({analysis['Overall_Trend']})\nRate: {analysis['Rate_of_Change']:.4f}")
    
    ax.set_title(f"{metric_name.replace('_', ' ')} Analysis")
    ax.set_xlabel('Time Period')
    ax.set_ylabel('Value')
    ax.legend()
    ax.grid(True)

plt.tight_layout()
plt.savefig('result/trend_analysis_plots.png')
plt.show()

print(f"Trend analysis has been saved to 'result/trend_analysis.csv'")
print(f"Visualization has been saved to 'result/trend_analysis_plots.png'")

# Display sample of the trend analysis
print("\nSample of trend analysis data:")
print(trend_df.head())